In [2]:
import numpy as np
import random
from mnist import MNIST
import matplotlib.pyplot as plt

In [3]:
mndata = MNIST('../HW1/Dataset')
trainX = np.array(mndata.load_training()[0])[:20000]
trainY = np.array(mndata.load_training()[1])[:20000]

testX = np.array(mndata.load_testing()[0])[:2000]
testY = np.array(mndata.load_testing()[1])[:2000]

In [4]:
#random selection of data for training and validation
valIndices = np.random.choice(len(trainX), 2000)
nonValIndices = [x for x in range(len(trainX)) if x not in valIndices]

valX = trainX[valIndices]
valY = trainY[valIndices]

trainX = trainX[nonValIndices]
trainY = trainY[nonValIndices]


def feat(data,i):
    return data[i].tolist()

def oneHot(clas, noOfClasses):
    feat = np.zeros(noOfClasses)
    feat[clas] = 1;
    return feat

trnX = np.array([feat(trainX,i) for i in range(trainX.shape[0])])/256.0
trnY = np.array([oneHot(trainY[i], 10) for i in range(trainX.shape[0])])

tstX = np.array([feat(testX,i) for i in range(testX.shape[0])])/256.0
tstY = np.array([oneHot(testY[i], 10) for i in range(testX.shape[0])])

valX = np.array([feat(valX,i) for i in range(valX.shape[0])])/256.0
valY = np.array([oneHot(valY[i], 10) for i in range(valX.shape[0])])

In [5]:
trnY[0:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
def sigmoid(x):
    return 1.0/(1+np.exp(-1*np.array(x)))

def grad(x):
    return x*(1-x)

def softmax(x):
    x = np.exp(x)
    x = x/x.sum(axis=1)[:, None]
    return x
    
np.random.seed(1)
lr = 0.0001

# randomly initialize our weights with mean 0
n_hid_1 = 4
W1 = 2*np.random.random((784,n_hid_1)) - 1
bias = 2*np.random.random((n_hid_1)) - 1
W2 = 2*np.random.random((n_hid_1,10)) - 1
bias2 = 2*np.random.random((10)) - 1

for j in xrange(1000):
    A1 = np.dot(trnX, W1) + bias
    l1 = sigmoid(A1)
    
    A2 = np.dot(l1, W2) + bias2
    l2 = softmax(A2)

    # Errors in output layer
    d2 = (l2 - trnY)
    dbias2 = np.sum(d2, axis = 0)
        
    # Delta of W2
    dW2 = np.dot(l1.T, d2)

    # Errors in 1st hidden layer
    d1 = np.dot(d2, W2.T)*grad(l1)
    dbias = np.sum(d1, axis = 0)    
    
    # Delta W2
    dW1 = np.dot(trnX.T, d1)
    
    W2 -= lr*dW2
    bias2 -= lr*dbias2
    W1 -= lr*dW1
    bias -= lr*dbias
    
    prediction = softmax(np.dot(sigmoid(np.dot(tstX, W1)+bias ), W2)+bias2)
    correct = [1 if a == b else 0 for (a, b) in zip(np.argmax(tstY, axis = 1), np.argmax(prediction, axis = 1))]
    
    print np.sum(correct)*100.0/len(tstX)

14.5
14.85
13.9
13.1
18.75
20.7
22.15
23.45
24.85
25.45
25.75
26.5
27.1
27.75
28.2
28.3
28.6
29.05
29.5
30.15
30.95
31.55
31.95
32.4
32.8
33.35
33.95
35.1
35.45
36.2
36.8
37.85
38.65
39.85
40.2
40.5
40.8
41.0
41.55
41.65
41.9
42.4
43.9
43.95
44.05
44.2
44.5
45.05
45.45
45.45
45.75
45.7
46.15
46.25
46.4
46.65
46.8
46.8
47.05
47.2
47.45
47.5
47.75
47.95
48.0
48.25
48.45
48.55
48.6
48.8
48.85
49.2
49.4
49.7
50.05
50.35
50.75
51.15
51.25
51.65
51.9
52.25
52.6
52.85
53.1
53.5
53.9
54.65
55.1
55.75
56.0
56.3
56.65
56.75
56.9
56.95
57.2
57.4
57.65
57.8
57.75
57.75
58.05
58.25
58.4
58.4
58.35
58.35
58.4
58.5
58.6
58.7
59.05
59.2
59.45
59.55
59.75
59.95
60.25
60.3
60.55
60.65
60.85
60.95
61.15
61.3
61.45
61.55
61.55
61.65
61.65
61.8
61.9
61.95
61.95
62.1
62.15
62.25
62.2
62.15
62.25
62.3
62.3
62.3
62.3
62.55
62.55
62.65
62.75
62.85
63.0
63.25
63.45
63.6
63.7
63.75
63.7
63.7
63.8
63.9
63.95
64.0
64.0
64.15
64.25
64.4
64.5
64.5
64.65
64.65
64.7
64.65
64.7
64.6
64.6
64.55
64.6
64.6
64.65
64.8
65.0

In [125]:
A2

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])